In [ ]:
from tunable_agents import utility, main
import utils
import gin
import numpy as np
import tensorflow as tf
from tf_agents.environments import tf_py_environment
from tf_agents.policies import py_tf_eager_policy
from tunable_agents.environments.gathering_env import gathering_env
import matplotlib.pyplot as plt

%matplotlib inline
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
absolute_path = "C:/Users/maler/Federico/Università/Master/Tesi"
configs_dir = "C:/Users/maler/Google Drive/Personale/Università/Master/Tesi/Code/tunable-agents-MORL/configs/"
root_dir = absolute_path + "/experiments_results"
experiment_name = "gathering_longer_epochs"
policy_dir = os.path.join(root_dir, experiment_name, "policy")

gin_files = [configs_dir + "envs/gathering_replication_env.gin"]
gin_files = [configs_dir + "envs/gathering_fixed_env.gin"]
gin_bindings = []
utility.load_gin_configs(gin_files, gin_bindings)

In [ ]:
env = gathering_env.GatheringWrapper(utility_repr=np.array([1,5,3,4,5,6], dtype=np.float32))
env = gathering_env.GatheringWrapper()
#env = gathering_env.GatheringWrapper(cumulative_rewards_flag=True)

eager_py_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
    policy_dir, env.time_step_spec(), env.action_spec())

In [ ]:
time_step = env.reset()
action = None

In [ ]:
plt.figure(figsize=(7,7), dpi=150)
image = time_step.observation['state_obs'][:, :, -3:]
plt.imshow(image)
plt.title('Utility: {}    Action: {}'.format(time_step.reward, action), fontsize=5)
plt.axis('off')
plt.show()

In [ ]:
# 0: Do nothing
# 1: Do nothing
# 2: Do nothing
# 3: Do nothing
# 4: Do nothing
action = 

time_step = tf_env.step(action)